In [1]:
# Importación de librerías necesarias
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import logging
import warnings
import sys
#
warnings.filterwarnings("ignore")
# Configurar logging y warnings
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')
set_log_level("ERROR")

In [2]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')
productos_descripcion = pd.read_csv('data/tb_productos_descripcion.txt', delimiter='\t')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1600)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())
print(productos_descripcion.head())
productos_descripcion.info()

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_

In [3]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [4]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
filtered_data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(filtered_data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = filtered_data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = filtered_data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
filtered_data.info()
filtered_data

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [5]:

# 2. Exploración inicial de los datos
print("Información de filtered_data:")
print(filtered_data.info())
print("\nInformación de productos_descripcion:")
print(productos_descripcion.info())

# 3. Fusión de los dataframes
df_merged = pd.merge(filtered_data, productos_descripcion, on='product_id', how='left')

# 4. Feature Engineering

# 4.1 Características temporales
df_merged['periodo'] = pd.to_datetime(df_merged['periodo'])
df_merged['mes'] = df_merged['periodo'].dt.month
df_merged['trimestre'] = df_merged['periodo'].dt.quarter

# 4.2 Codificación de variables categóricas
le = LabelEncoder()
df_merged['cat1_encoded'] = le.fit_transform(df_merged['cat1'])
df_merged['cat2_encoded'] = le.fit_transform(df_merged['cat2'])
df_merged['cat3_encoded'] = le.fit_transform(df_merged['cat3'])
df_merged['brand_encoded'] = le.fit_transform(df_merged['brand'])

# 4.3 Características basadas en agrupaciones
df_merged['avg_tn_por_producto'] = df_merged.groupby('product_id')['tn'].transform('mean')
df_merged['avg_qty_por_cliente'] = df_merged.groupby('customer_id')['cust_request_qty'].transform('mean')

# 4.4 Ratios y proporciones
df_merged['ratio_tn_qty'] = df_merged['tn'] / df_merged['cust_request_qty']

# 4.5 Flags binarios
df_merged['is_precios_cuidados'] = df_merged['plan_precios_cuidados'].map({'Si': 1, 'No': 0})

# 4.6 Uso directo del tamaño del SKU
df_merged['sku_size_numeric'] = df_merged['sku_size'].fillna(0).astype(float)

# 4.7 Categorización del tamaño del SKU
df_merged['sku_size_category'] = pd.cut(df_merged['sku_size'], 
                                        bins=[0, 100, 250, 500, 1000, float('inf')],
                                        labels=['Muy pequeño', 'Pequeño', 'Mediano', 'Grande', 'Muy grande'])

df_merged['sku_size_category_encoded'] = le.fit_transform(df_merged['sku_size_category'])

# 5. Selección de variables para NeuralProphet
regresores = [
    'mes', 'dia_semana', 'trimestre', 'cat1_encoded', 'cat2_encoded', 'cat3_encoded',
    'brand_encoded', 'avg_tn_por_producto', 'avg_qty_por_cliente', 'ratio_tn_qty',
    'is_precios_cuidados', 'sku_size_numeric', 'sku_size_category_encoded'
]

# 6. Preparación del dataframe final para NeuralProphet
df_final = df_merged[['periodo', 'tn'] + regresores].rename(columns={'periodo': 'ds', 'tn': 'y'})

# 7. Guardado del dataframe final
df_final.to_csv('df_final_para_neuralprophet.csv', index=False)

print("Dataframe final guardado con éxito. Listo para ser utilizado en NeuralProphet.")

# 8. Resumen de las nuevas características
print("\nResumen de las nuevas características:")
for column in regresores:
    if df_final[column].dtype == 'object':
        print(f"\n{column}:")
        print(df_final[column].value_counts())
    else:
        print(f"\n{column}:")
        print(df_final[column].describe())

# 9. Verificación de valores nulos
print("\nVerificación de valores nulos:")
print(df_final.isnull().sum())

# 10. Correlaciones entre las variables
print("\nCorrelaciones entre las variables:")
correlation_matrix = df_final[['y'] + regresores].corr()
print(correlation_matrix['y'].sort_values(ascending=False))

df_final

Información de filtered_data:
<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 140.0 MB
None

Información de productos_descripcion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   cat1         1251 non-null   object
 1   cat2         1251 non-null   object
 2   cat3         1251 non-null   object
 3   brand        1251 non-null   object
 4   sku_size     1251 non-null   int64 
 5   product_id   1251 non-null   int64 
 6   desc

KeyError: "['dia_semana'] not in index"

In [ ]:
# Calcular diferentes customer_id y product_id
num_customers = filtered_data['customer_id'].nunique()
num_products = filtered_data['product_id'].nunique()

print(f'Número de diferentes customer_id: {num_customers}')
print(f'Número de diferentes product_id: {num_products}')

In [ ]:
# Crear un listado por período, customer_id y product_id
listado = filtered_data.groupby(['periodo', 'customer_id', 'product_id']).size().reset_index(name='num_pedidos')

# Ordenar por período y customer_id
listado_ordenado = listado.sort_values(by=['periodo', 'customer_id'])

# Mostrar el resultado
print(listado_ordenado)

In [ ]:
# Filtrar los registros donde num_pedidos sea mayor que 1
registros_multiples_pedidos = listado_ordenado[listado_ordenado['num_pedidos'] > 1]

# Mostrar el resultado
print(registros_multiples_pedidos)

In [ ]:
# Crear un listado por período y product_id sumando todos los clientes
listado_productos_periodos = filtered_data.groupby(['periodo', 'product_id']).size().reset_index(name='num_pedidos')

# Contar el total de periodos para cada product_id
total_periodos_por_producto = listado_productos_periodos.groupby('product_id')['periodo'].nunique().reset_index(name='total_periodos')

# Ordenar por total de periodos en orden descendente
total_periodos_por_producto_ordenado = total_periodos_por_producto.sort_values(by='total_periodos', ascending=False)

# Configurar pandas para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Mostrar el resultado completo
#print(total_periodos_por_producto_ordenado)

In [ ]:
# Asegurarse de que la columna 'periodo' esté en formato de fecha
filtered_data['periodo'] = pd.to_datetime(filtered_data['periodo'], format='%Y%m')

# Agrupar por período y sumar la cantidad de ventas en toneladas
ventas_por_periodo = filtered_data.groupby(filtered_data['periodo'].dt.to_period('M'))['tn'].sum().reset_index(name='total_toneladas')

# Convertir la columna 'periodo' de nuevo a formato de fecha
ventas_por_periodo['periodo'] = ventas_por_periodo['periodo'].dt.to_timestamp()

# Crear el gráfico de barras en color verde
plt.figure(figsize=(12, 6))
plt.bar(ventas_por_periodo['periodo'], ventas_por_periodo['total_toneladas'], color='green', width=20)  # Ajusta el ancho de las barras

# Agregar títulos y etiquetas
plt.title('Cantidad de ventas en toneladas por período')
plt.xlabel('Período')
plt.ylabel('Total de toneladas vendidas')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Mostrar el gráfico
plt.tight_layout()
plt.show()

# Mostrar el listado de los periodos con las toneladas vendidas
print(ventas_por_periodo)

In [ ]:
# Asegurarse de que la columna 'periodo' esté en formato de fecha
filtered_data['periodo'] = pd.to_datetime(filtered_data['periodo'], format='%Y%m')

# Agrupar por período y contar la cantidad de productos vendidos en cada mes
ventas_por_mes = filtered_data.groupby(filtered_data['periodo'].dt.to_period('M')).size().reset_index(name='cantidad_productos')

# Convertir la columna 'periodo' de nuevo a formato de fecha
ventas_por_mes['periodo'] = ventas_por_mes['periodo'].dt.to_timestamp()

# Crear el gráfico
plt.figure(figsize=(12, 6))
plt.plot(ventas_por_mes['periodo'], ventas_por_mes['cantidad_productos'], marker='o', linestyle='-', color='b')

# Agregar títulos y etiquetas
plt.title('Cantidad de productos vendidos por mes')
plt.xlabel('Mes')
plt.ylabel('Cantidad de productos vendidos')
plt.grid(True)

# Mostrar el gráfico
plt.show()

In [ ]:
# Asegurarse de que la columna 'periodo' esté en formato de fecha
filtered_data['periodo'] = pd.to_datetime(filtered_data['periodo'], format='%Y%m')

# Agrupar por período y contar la cantidad de distintos productos vendidos en cada período
productos_por_periodo = filtered_data.groupby(filtered_data['periodo'].dt.to_period('M'))['product_id'].nunique().reset_index(name='cantidad_productos')

# Convertir la columna 'periodo' de nuevo a formato de fecha
productos_por_periodo['periodo'] = productos_por_periodo['periodo'].dt.to_timestamp()

# Crear el gráfico de barras con barras más gruesas
plt.figure(figsize=(12, 6))
plt.bar(productos_por_periodo['periodo'], productos_por_periodo['cantidad_productos'], color='skyblue', width=20)  # Ajusta el ancho de las barras

# Agregar títulos y etiquetas
plt.title('Cantidad de distintos productos vendidos por período')
plt.xlabel('Período')
plt.ylabel('Cantidad de distintos productos vendidos')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Mostrar el gráfico
plt.tight_layout()
plt.show()

# Mostrar el listado de los 36 meses con los resultados de diferentes productos vendidos
print(productos_por_periodo)

In [ ]:
#  Version 7.1.1
#  Se agrega separan los productos que tienen mas de 20 meses de historia para ser tratados diferente en el algoritmo
#
# Función para filtrar datos por rango de product_id
def filtrar_por_producto(df, producto_inicio, producto_fin):
    return df[(df['product_id'] >= producto_inicio) & (df['product_id'] <= producto_fin)]

# Función para preparar datos para NeuralProphet
def preparar_datos(df):
    df = df.groupby(['periodo', 'product_id']).agg({'tn': 'sum'}).reset_index()
    df['ds'] = pd.to_datetime(df['periodo'], format='%Y%m')
    df.rename(columns={'tn': 'y'}, inplace=True)
    return df[['ds', 'y', 'product_id']]

# Filtrado de datos para el rango de productos deseado
producto_inicio = 20001
producto_fin = 20012
#producto_fin = 21276
datos_filtrados = filtrar_por_producto(filtered_data, producto_inicio, producto_fin)

# Preparación de datos para NeuralProphet
datos = preparar_datos(datos_filtrados)
datos.head()
datos.info()

In [ ]:
#Nos aseguramos de tener registros en cero para productos que pudieron no haber tenido ventas en un periodo a pesar de seguir estando en stock

# Asegurémonos de que los datos estén ordenados por product_id y ds
datos['ds'] = pd.to_datetime(datos['ds'])
datos.sort_values(by=['product_id', 'ds'], inplace=True)

# Crear una función para agregar registros en cero
def agregar_registros_cero(df):
    df_list = []
    for producto, datos_producto in df.groupby('product_id'):
        # Obtener el rango de fechas completo para el producto
        fecha_inicio = datos_producto['ds'].min()
        fecha_fin = datos_producto['ds'].max()
        rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='MS')  # Frecuencia mensual

        # Completar el dataframe con fechas faltantes y valores en cero
        datos_completos = pd.DataFrame(rango_fechas, columns=['ds'])
        datos_completos['product_id'] = producto
        datos_completos = datos_completos.merge(datos_producto, on=['ds', 'product_id'], how='left')
        datos_completos['y'].fillna(0, inplace=True)  # Llenar NaNs con ceros

        df_list.append(datos_completos)

    return pd.concat(df_list, ignore_index=True)

# Mostrar cantidad de registros antes de agregar ceros
print(f"Cantidad de registros antes de agregar ceros: {len(datos)}")

# Aplicar la función para agregar registros en cero
datos_completos_cero = agregar_registros_cero(datos)

# Mostrar cantidad de registros después de agregar ceros
print(f"Cantidad de registros después de agregar ceros: {len(datos_completos_cero)}")

# Mostrar los primeros registros del dataframe resultante
print("\nPrimeros registros del dataframe resultante:")
print(datos_completos_cero.head())

In [ ]:
# Primero, veamos qué columnas tienes en tu DataFrame
print("Columnas en el DataFrame:")
print(datos.columns)

In [ ]:
# Evaluamos los modelos que tienen mas de 15 periodos en el data frame
#
# Suponemos que 'datos' ya está preparado y contiene las columnas ['ds', 'y', 'product_id']
datos['ds'] = pd.to_datetime(datos['ds'])
datos.sort_values(by=['product_id', 'ds'], inplace=True)

# Dividir los datos en train, validation y test
train_df = datos_completos_cero[datos_completos_cero['ds'] < '2019-11-01']
val_df = datos_completos_cero[datos_completos_cero['ds'] == '2019-11-01']
test_df = datos_completos_cero[datos_completos_cero['ds'] == '2019-12-01']

# Filtrar productos con más de 15 periodos de historia
productos_mas_historia = datos_completos_cero['product_id'].value_counts()[datos_completos_cero['product_id'].value_counts() > 15].index.tolist()
print(f"Número de productos con más de 15 periodos de historia: {len(productos_mas_historia)}")

def entrenar_y_predecir(train_df, val_df, test_df):
    modelos = {}
    predicciones = pd.DataFrame()
    for producto in tqdm(productos_mas_historia, desc="Procesando productos"):
        df_producto_train = train_df[train_df['product_id'] == producto][['ds', 'y']]
        df_producto_val = val_df[val_df['product_id'] == producto][['ds', 'y']]
        df_producto_test = test_df[test_df['product_id'] == producto][['ds', 'y']]
        
        m = NeuralProphet(
            n_forecasts=2,
            yearly_seasonality=True,
            batch_size=12,
            epochs=1000,
            learning_rate=0.025,
            loss_func='Huber',
            daily_seasonality=False,
            weekly_seasonality=False,
            seasonality_mode='multiplicative',
            lagged_reg_layers=[16, 8],  # Ajustar las capas ocultas
            normalize="standardize"
        )
        
        # Combinar train y validation para el entrenamiento final
        df_producto_train_val = pd.concat([df_producto_train, df_producto_val])
        
        m.fit(df_producto_train, validation_df=df_producto_val, freq='M')
        
        # Crear future dataframe manualmente
        last_date = df_producto_test['ds'].max()
        future_dates = pd.date_range(start=last_date, periods=3, freq='MS')[1:]  # Dos meses adelante
        future = pd.DataFrame({'ds': future_dates, 'y': [None]*len(future_dates)})
        
        forecast = m.predict(future)
        
        forecast.loc[forecast['yhat1'] < 0, 'yhat1'] = 0  # Convertir valores negativos a cero
        pred_febrero_2020 = forecast[forecast['ds'] == pd.Timestamp('2020-02-01')][['ds', 'yhat1']]
        pred_febrero_2020['product_id'] = producto
        predicciones = pd.concat([predicciones, pred_febrero_2020[['ds', 'yhat1', 'product_id']]])
        modelos[producto] = m
    return predicciones, modelos

# Entrenar el modelo y obtener predicciones
predicciones_febrero_2020, modelos = entrenar_y_predecir(train_df, val_df, test_df)

# Renombrar las columnas para el archivo de salida
predicciones_febrero_2020.rename(columns={'yhat1': 'tn'}, inplace=True)

# Convertir la columna 'ds' al formato YYYYMM
predicciones_febrero_2020['ds'] = predicciones_febrero_2020['ds'].dt.strftime('%Y%m')

# Guardar el resultado en un archivo .csv solo con la predicción para febrero 2020
predicciones_febrero_2020.to_csv('NeuralProphet_mas_20_periodos.v7.1.1.csv', index=False)

# Mostrar un resumen del resultado
print(f"Número total de predicciones: {len(predicciones_febrero_2020)}")
print(predicciones_febrero_2020.head())

In [ ]:
# Suponemos que 'datos' ya está preparado y contiene las columnas ['ds', 'y', 'product_id']
datos['ds'] = pd.to_datetime(datos['ds'])
datos.sort_values(by=['product_id', 'ds'], inplace=True)

# Dividir los datos en train y test
train_df = datos_completos_cero[datos_completos_cero['ds'] < '2019-12-01']
test_df = datos_completos_cero[datos_completos_cero['ds'] == '2019-12-01']

# Filtrar productos con menos o igual a 15 periodos de historia
productos_menos_historia = datos_completos_cero['product_id'].value_counts()[datos_completos_cero['product_id'].value_counts() <= 15].index.tolist()
print(f"Número de productos con menos o igual a 15 periodos de historia: {len(productos_menos_historia)}")

def entrenar_y_predecir_menos_historia(train_df, test_df):
    modelos = {}
    predicciones = pd.DataFrame()
    for producto in tqdm(productos_menos_historia, desc="Procesando productos"):
        df_producto_train = train_df[train_df['product_id'] == producto][['ds', 'y']]
        df_producto_test = test_df[test_df['product_id'] == producto][['ds', 'y']]
        
        if len(df_producto_train) < 5:
            promedio = df_producto_train['y'].mean()
            pred_febrero_2020 = pd.DataFrame({
                'ds': [pd.Timestamp('2020-02-01')],
                'yhat1': [promedio],
                'product_id': [producto]
            })
            predicciones = pd.concat([predicciones, pred_febrero_2020])
        else:
            m = NeuralProphet(
                n_forecasts=2,
                yearly_seasonality=True,
                batch_size=6,
                epochs=1000,
                learning_rate=0.025,
                loss_func='Huber',
                daily_seasonality=False,
                weekly_seasonality=False,
                seasonality_mode='multiplicative',
                lagged_reg_layers=[16, 8],  # Ajustar las capas ocultas
                normalize="standardize",
            )
            
            m.fit(df_producto_train, freq='M')
            
            last_date = df_producto_test['ds'].max()
            future_dates = pd.date_range(start=last_date, periods=3, freq='MS')[1:]
            future = pd.DataFrame({'ds': future_dates, 'y': [None]*len(future_dates)})
            
            forecast = m.predict(future)
            
            forecast.loc[forecast['yhat1'] < 0, 'yhat1'] = 0
            pred_febrero_2020 = forecast[forecast['ds'] == pd.Timestamp('2020-02-01')][['ds', 'yhat1']]
            pred_febrero_2020['product_id'] = producto
            predicciones = pd.concat([predicciones, pred_febrero_2020[['ds', 'yhat1', 'product_id']]])
            modelos[producto] = m
    
    return predicciones, modelos

# Entrenar el modelo y obtener predicciones
predicciones_febrero_2020, modelos = entrenar_y_predecir_menos_historia(train_df, test_df)

# Verificar si se generaron predicciones
if predicciones_febrero_2020.empty:
    print("No se generaron predicciones. Verifica los datos de entrada.")
else:
    print(f"Número total de predicciones: {len(predicciones_febrero_2020)}")
    
    # Renombrar las columnas para el archivo de salida
    if 'yhat1' in predicciones_febrero_2020.columns:
        predicciones_febrero_2020.rename(columns={'yhat1': 'tn'}, inplace=True)
    
    # Convertir la columna 'ds' al formato YYYYMM si existe
    if 'ds' in predicciones_febrero_2020.columns:
        predicciones_febrero_2020['ds'] = pd.to_datetime(predicciones_febrero_2020['ds']).dt.strftime('%Y%m')
    
    # Guardar el resultado en un archivo .csv solo con la predicción para febrero 2020
    predicciones_febrero_2020.to_csv('NeuralProphet_menos_20_periodos.v7.1.1.csv', index=False)
    
    # Mostrar un resumen del resultado
    print(predicciones_febrero_2020.head())

In [ ]:
# Leer los archivos CSV
df_menos = pd.read_csv('NeuralProphet_menos_20_periodos.v7.1.1.csv')
df_mas = pd.read_csv('NeuralProphet_mas_20_periodos.v7.1.1.csv')

# Combinar los dos DataFrames
df_combinado = pd.concat([df_menos, df_mas])

# Ordenar por product_id
df_combinado = df_combinado.sort_values('product_id')

# Seleccionar solo las columnas 'product_id' y 'tn'
df_final = df_combinado[['product_id', 'tn']]

# Guardar el resultado en un nuevo archivo CSV
df_final.to_csv('NeuralProphet.v7.1.1.CAN.paraKaggle.csv', index=False)

print("Archivo combinado creado con éxito.")

In [ ]:
# Leer el archivo CSV
df = pd.read_csv('NeuralProphet.v7.1.1.CAN.paraKaggle.csv')

# Calcular la suma total actual de 'tn'
suma_actual = df['tn'].sum()
print(f"Suma total de 'tn': {suma_actual:.2f}")

# Calcular el factor de ajuste
factor_ajuste = 30600 / suma_actual

# Ajustar los valores de 'tn'
df['tn'] = df['tn'] * factor_ajuste

# Verificar que la nueva suma sea 30600
nueva_suma = df['tn'].sum()
print(f"Nueva suma total de 'tn': {nueva_suma:.2f}")

# Guardar el resultado en un nuevo archivo CSV
df.to_csv('NeuralProphet.v7.1.1.CAN.paraKaggle_ajustado.csv', index=False)

print("Archivo ajustado creado con éxito.")